In [51]:
import pandas as pd
import numpy as np
from datetime import datetime
import itertools
import re

trainDF = pd.read_csv("train.csv")
trafoDF = pd.read_csv("trafo.csv")
submissionDF = pd.read_csv("submission.csv")
bagilNemDF = pd.read_csv("Hava Durumu/Bağıl Nem.csv")
bulutsuzlukOraniDF = pd.read_csv("Hava Durumu/Bulutluluk Oranı.csv")
radyasyonDF = pd.read_csv("Hava Durumu/Radyasyon.csv")
ruzgarHiziDF = pd.read_csv("Hava Durumu/Rüzgar Hızı.csv")
ruzgarYonuDF = pd.read_csv("Hava Durumu/Rüzgar Yönü.csv")
sicaklikDF = pd.read_csv("Hava Durumu/Sıcaklık.csv")
yagisDF = pd.read_csv("Hava Durumu/Yağış.csv")

radyasyonDF.drop("Unnamed: 17",axis=1,inplace=True) # bilinmeyen kolon ve içeriği boş olduğu için drop ediyoruz
trainDF.drop("BİLDİRİME_GÖRE",axis=1,inplace=True) # Bu kolon için tüm degerler aynı,bir katkısı olmayacak bu sebeple drop ediyoruz
trainDF.drop(["KOD_NO","KADEME"],axis=1,inplace=True) # Bu kolonların için tüm degerleri null,bir katkısı olmayacak bu sebeple drop ediyoruz
trainDF.drop("SONA_ERME_TARİHİ_VE_ZAMANI",axis=1,inplace=True) # Bitiş tarihine gerek duluymadığı için drop ediyoruz
trainDF.drop("SEBEBE_GÖRE",axis=1,inplace=True) # kolon verilerinin %98'i "Şebeke işletmecisi" türünden olduğu için bu kolonu çıkarıyoruz

def tr_capitalize(param_word):
    word_list = param_word.split(sep=" ")
    new_word = ""
    for word in word_list:
        first_letter = word[0]
        restOfWord = word[1:]

        first_letter = re.sub(r"i", "İ", first_letter)
        first_letter = re.sub(r"ı", "I", first_letter)
        first_letter = re.sub(r"ç", "Ç", first_letter)
        first_letter = re.sub(r"ş", "Ş", first_letter)
        first_letter = re.sub(r"ü", "Ü", first_letter)
        first_letter = re.sub(r"ğ", "Ğ", first_letter)

        restOfWord = re.sub(r"İ", "i", restOfWord)
        restOfWord = re.sub(r"I", "ı", restOfWord)
        restOfWord = re.sub(r"Ç", "ç", restOfWord)
        restOfWord = re.sub(r"Ş", "ş", restOfWord)
        restOfWord = re.sub(r"Ü", "ü", restOfWord)
        restOfWord = re.sub(r"Ğ", "ğ", restOfWord)

        rebuilt_word = first_letter + restOfWord
        rebuilt_word = rebuilt_word.capitalize()
        new_word = new_word + " " + rebuilt_word

    new_word = new_word.strip()
    return new_word

    
trainDF["BAŞLAMA_TARİHİ"] = pd.to_datetime(trainDF["BAŞLAMA_TARİHİ_VE_ZAMANI"], dayfirst=True).dt.date

bagilNemDF["Tarih"]=pd.to_datetime(bagilNemDF["Tarih"], dayfirst=True,errors='coerce').dt.date
radyasyonDF["Tarih"]=pd.to_datetime(radyasyonDF["Tarih"], dayfirst=True,errors='coerce').dt.date
bulutsuzlukOraniDF["Tarih"]=pd.to_datetime(bulutsuzlukOraniDF["Tarih"], dayfirst=True,errors='coerce').dt.date
ruzgarHiziDF["Tarih"]=pd.to_datetime(ruzgarHiziDF["Tarih"], dayfirst=True,errors='coerce').dt.date
ruzgarYonuDF["Tarih"]=pd.to_datetime(ruzgarYonuDF["Tarih"], dayfirst=True,errors='coerce').dt.date
sicaklikDF["Tarih"]=pd.to_datetime(sicaklikDF["Tarih"], dayfirst=True,errors='coerce').dt.date
yagisDF["Tarih"]=pd.to_datetime(yagisDF["Tarih"], dayfirst=True,errors='coerce').dt.date


minDate = trainDF["BAŞLAMA_TARİHİ"].min()

# hava durumu ile ilgili dataframelerin tarihini traindf nin en düşük tarihine göre filtreliyoruz
bagilNemDF=bagilNemDF[bagilNemDF["Tarih"] >= minDate]
bulutsuzlukOraniDF=bulutsuzlukOraniDF[bulutsuzlukOraniDF["Tarih"] >= minDate]
radyasyonDF=radyasyonDF[radyasyonDF["Tarih"] >= minDate]
ruzgarHiziDF=ruzgarHiziDF[ruzgarHiziDF["Tarih"] >= minDate]
ruzgarYonuDF=ruzgarYonuDF[ruzgarYonuDF["Tarih"] >= minDate]
sicaklikDF=sicaklikDF[sicaklikDF["Tarih"] >= minDate]
yagisDF=yagisDF[yagisDF["Tarih"] >= minDate]

trainDF['SÜREYE_GÖRE']=trainDF['SÜREYE_GÖRE'].map({"Kısa":1,"Uzun":2}).astype(int)




In [52]:
# %80 ve üzeri oranda aynı degeri taşıyan kolonları siliyoruz
#?satirSayisi = len(finalDF[finalDF["KESİNTİ_NO"].notna()])
#! trainDF["TOPLAM_KENTALTI_OG"].value_counts()/satirSayisi #%94 0
#! trainDF["KENTSEL_OG"].value_counts()/satirSayisi # %80 0
#? trainDF["KENTSEL_AG"].value_counts()/satirSayisi # +++
#! trainDF["KIRSAL_OG"].value_counts()/satirSayisi# %92 0
#! trainDF["KIRSAL_AG"].value_counts()/satirSayisi# %82.5 0
#! trainDF["TOPLAM_KENTSEL_OG"].value_counts()/satirSayisi # %80 0
#? trainDF["TOPLAM_KENTSEL_AG"].value_counts()/satirSayisi # +++
#! trainDF["TOPLAM_KENTALTI_AG"].value_counts()/satirSayisi # %83 0
#! trainDF["TOPLAM_KIRSAL_OG"].value_counts()/satirSayisi # %920
#! trainDF["TOPLAM_KIRSAL_AG"].value_counts()/satirSayisi # %82 0

trainDF.drop("TOPLAM_KENTALTI_OG",axis=1,inplace=True)
trainDF.drop("KENTSEL_OG",axis=1,inplace=True)
trainDF.drop("KIRSAL_OG",axis=1,inplace=True)
trainDF.drop("KIRSAL_AG",axis=1,inplace=True)
trainDF.drop("TOPLAM_KENTSEL_OG",axis=1,inplace=True)
trainDF.drop("TOPLAM_KENTALTI_AG",axis=1,inplace=True)
trainDF.drop("TOPLAM_KIRSAL_OG",axis=1,inplace=True)
trainDF.drop("TOPLAM_KIRSAL_AG",axis=1,inplace=True)





In [53]:
havaDurumuLokasyonlar = list(set(itertools.chain(bagilNemDF.columns,bulutsuzlukOraniDF.columns,radyasyonDF.columns,ruzgarHiziDF.columns,ruzgarYonuDF.columns,sicaklikDF.columns,yagisDF.columns)))

trainDF["İL"]=trainDF["İL"].apply(lambda x: tr_capitalize(x))
trainDF["İLÇE"]=trainDF["İLÇE"].apply(lambda x: tr_capitalize(x))
trainDF["LOKASYON"] = trainDF[["İL","İLÇE"]].apply(lambda x: x["İLÇE"] if x["İLÇE"] in havaDurumuLokasyonlar else x["İL"], axis=1)
trainDF.drop(["İL","İLÇE"],axis=1,inplace=True)

In [54]:
trainDF=pd.merge(trainDF,trafoDF,on='ŞEBEKE_UNSURU_KODU',how='left')
trainDF['target']=1
#start -> trainDF["BAŞLAMA_TARİHİ"]).min() and end -> trainDF["BAŞLAMA_TARİHİ"]).max()
dframe=pd.DataFrame(pd.date_range(start='5/1/2021', end='6/12/2022', freq='D'))
dframe.rename(columns={0:"Tarih"},inplace=True)

uniqueTrafo=pd.DataFrame()
uniqueTrafo["trafo_id"]=trainDF["trafo_id"].unique()
# uniqueTrafo=uniqueTrafo.merge(trainDF[["trafo_id","LOKASYON"]],on="trafo_id",how="inner").drop_duplicates()
# dframe=dframe.merge(uniqueTrafo[["trafo_id","LOKASYON"]], how='cross')

uniqueTrafo=uniqueTrafo.merge(trainDF["trafo_id"],on="trafo_id",how="inner").drop_duplicates()
dframe=dframe.merge(uniqueTrafo["trafo_id"], how='cross')

trainDF.rename(columns={"BAŞLAMA_TARİHİ":"Tarih"},inplace=True)
trainDF["Tarih"]=trainDF["Tarih"].astype('datetime64[ns]')


trafo_lokasyon=trainDF[["trafo_id","LOKASYON"]].drop_duplicates()
finalDF = pd.merge(dframe,trafo_lokasyon, on=['trafo_id'],how="left")

finalDF

,Tarih,trafo_id,LOKASYON
0,2021-05-01,0,Urla
1,2021-05-01,1,İzmir
2,2021-05-01,2,İzmir
3,2021-05-01,3,Bergama
4,2021-05-01,4,İzmir
...,...,...,...
26007139,2022-06-12,63762,İzmir
26007140,2022-06-12,63763,İzmir
26007141,2022-06-12,63764,İzmir
26007142,2022-06-12,63765,Akhisar


In [55]:
sebekeUnsuru_TrafoID=trainDF[["trafo_id","ŞEBEKE_UNSURU"]].drop_duplicates()
finalDF = pd.merge(finalDF,sebekeUnsuru_TrafoID, on=['trafo_id'],how="left")
trainDF.drop("ŞEBEKE_UNSURU",axis=1,inplace=True) # finalDF ye eklendiği için drop ediyoruz
finalDF

,Tarih,trafo_id,LOKASYON,ŞEBEKE_UNSURU
0,2021-05-01,0,Urla,OG Fideri
1,2021-05-01,1,İzmir,OG Fideri
2,2021-05-01,2,İzmir,TM Fideri
3,2021-05-01,3,Bergama,OG Fideri
4,2021-05-01,4,İzmir,OG Fideri
...,...,...,...,...
26007139,2022-06-12,63762,İzmir,Abone Tesisi
26007140,2022-06-12,63763,İzmir,Abone Tesisi
26007141,2022-06-12,63764,İzmir,Abone Tesisi
26007142,2022-06-12,63765,Akhisar,Abone Tesisi


In [56]:
bagilNem = bagilNemDF.melt(id_vars = 'Tarih', var_name = 'LOKASYON', value_name = 'BAĞIL_NEM').groupby(["Tarih","LOKASYON"]).mean().reset_index()
bagilNem["Tarih"]=pd.to_datetime(bagilNem["Tarih"], dayfirst=True)

bulutsuzlukOrani = bulutsuzlukOraniDF.melt(id_vars = 'Tarih', var_name = 'LOKASYON', value_name = 'BULUTSUZLUK_ORANI').groupby(["Tarih","LOKASYON"]).mean().reset_index()
bulutsuzlukOrani["Tarih"]=pd.to_datetime(bulutsuzlukOrani["Tarih"], dayfirst=True)

radyasyon = radyasyonDF.melt(id_vars = 'Tarih', var_name = 'LOKASYON', value_name = 'RADYASYON').groupby(["Tarih","LOKASYON"]).mean().reset_index()
radyasyon["Tarih"]=pd.to_datetime(radyasyon["Tarih"], dayfirst=True)

ruzgarHizi = ruzgarHiziDF.melt(id_vars = 'Tarih', var_name = 'LOKASYON', value_name = 'RUZGAR_HIZI').groupby(["Tarih","LOKASYON"]).mean().reset_index()
ruzgarHizi["Tarih"]=pd.to_datetime(ruzgarHizi["Tarih"], dayfirst=True)

ruzgarYonu = ruzgarYonuDF.melt(id_vars = 'Tarih', var_name = 'LOKASYON', value_name = 'RUZGAR_YONU').groupby(["Tarih","LOKASYON"]).mean().reset_index()
ruzgarYonu["Tarih"]=pd.to_datetime(ruzgarYonu["Tarih"], dayfirst=True)

sicaklik = sicaklikDF.melt(id_vars = 'Tarih', var_name = 'LOKASYON', value_name = 'SICAKLIK').groupby(["Tarih","LOKASYON"]).mean().reset_index()
sicaklik["Tarih"]=pd.to_datetime(sicaklik["Tarih"], dayfirst=True)

yagis = yagisDF.melt(id_vars = 'Tarih', var_name = 'LOKASYON', value_name = 'YAGIS').groupby(["Tarih","LOKASYON"]).mean().reset_index()
yagis["Tarih"]=pd.to_datetime(yagis["Tarih"], dayfirst=True)

havaDurumuDF=pd.merge(bagilNem,bulutsuzlukOrani, on=['Tarih',"LOKASYON"],how="outer")
havaDurumuDF=pd.merge(havaDurumuDF,radyasyon, on=['Tarih',"LOKASYON"],how="outer")
havaDurumuDF=pd.merge(havaDurumuDF,ruzgarHizi, on=['Tarih',"LOKASYON"],how="outer")
havaDurumuDF=pd.merge(havaDurumuDF,ruzgarYonu, on=['Tarih',"LOKASYON"],how="outer")
havaDurumuDF=pd.merge(havaDurumuDF,sicaklik, on=['Tarih',"LOKASYON"],how="outer")
havaDurumuDF=pd.merge(havaDurumuDF,yagis, on=['Tarih',"LOKASYON"],how="outer")


In [57]:
finalDF = pd.merge(finalDF,havaDurumuDF, on=['Tarih',"LOKASYON"],how="left")


In [58]:
sicaklik["yil"] = sicaklik["Tarih"].dt.year 
sicaklik["ay"] = sicaklik["Tarih"].dt.month 
 
sicaklik_dict = sicaklik.groupby(["yil","ay","LOKASYON"])["SICAKLIK"].mean().to_dict() 
 
finalDF["SICAKLIK"]  = finalDF.apply(lambda x : sicaklik_dict[(x["Tarih"].year,x["Tarih"].month, x["LOKASYON"])] if (pd.isna(x["SICAKLIK"])) else x["SICAKLIK"] ,axis=1)

In [59]:
bagilNem["yil"] = bagilNem["Tarih"].dt.year 
bagilNem["ay"] = bagilNem["Tarih"].dt.month 
 
bagilNem_dict = bagilNem.groupby(["yil","ay","LOKASYON"])["BAĞIL_NEM"].mean().to_dict() 
 
finalDF["BAĞIL_NEM"]  = finalDF.apply(lambda x : bagilNem_dict[(x["Tarih"].year,x["Tarih"].month, x["LOKASYON"])] if (pd.isna(x["BAĞIL_NEM"])) else x["BAĞIL_NEM"] ,axis=1)

In [60]:
bulutsuzlukOrani["yil"] = bulutsuzlukOrani["Tarih"].dt.year 
bulutsuzlukOrani["ay"] = bulutsuzlukOrani["Tarih"].dt.month 
 
bulutsuzlukOrani_dict = bulutsuzlukOrani.groupby(["yil","ay","LOKASYON"])["BULUTSUZLUK_ORANI"].mean().to_dict() 
 
finalDF["BULUTSUZLUK_ORANI"]  = finalDF.apply(lambda x : bulutsuzlukOrani_dict[(x["Tarih"].year,x["Tarih"].month, x["LOKASYON"])] if (pd.isna(x["BULUTSUZLUK_ORANI"])) else x["BULUTSUZLUK_ORANI"] ,axis=1)

In [61]:
radyasyon["yil"] = radyasyon["Tarih"].dt.year 
radyasyon["ay"] = radyasyon["Tarih"].dt.month 
 
radyasyon_dict = radyasyon.groupby(["yil","ay","LOKASYON"])["RADYASYON"].mean().to_dict() 
 
finalDF["RADYASYON"]  = finalDF.apply(lambda x : radyasyon_dict[(x["Tarih"].year,x["Tarih"].month, x["LOKASYON"])] if (pd.isna(x["RADYASYON"])) else x["RADYASYON"] ,axis=1)

In [62]:
ruzgarHizi["yil"] = ruzgarHizi["Tarih"].dt.year 
ruzgarHizi["ay"] = ruzgarHizi["Tarih"].dt.month 
 
ruzgarHizi_dict = ruzgarHizi.groupby(["yil","ay","LOKASYON"])["RUZGAR_HIZI"].mean().to_dict() 
 
finalDF["RUZGAR_HIZI"]  = finalDF.apply(lambda x : ruzgarHizi_dict[(x["Tarih"].year,x["Tarih"].month, x["LOKASYON"])] if (pd.isna(x["RUZGAR_HIZI"])) else x["RUZGAR_HIZI"] ,axis=1)

In [63]:
ruzgarYonu["yil"] = ruzgarYonu["Tarih"].dt.year 
ruzgarYonu["ay"] = ruzgarYonu["Tarih"].dt.month 
 
ruzgarYonu_dict = ruzgarYonu.groupby(["yil","ay","LOKASYON"])["RUZGAR_YONU"].mean().to_dict() 
 
finalDF["RUZGAR_YONU"]  = finalDF.apply(lambda x : ruzgarYonu_dict[(x["Tarih"].year,x["Tarih"].month, x["LOKASYON"])] if (pd.isna(x["RUZGAR_YONU"])) else x["RUZGAR_YONU"] ,axis=1)

In [64]:
yagis["yil"] = yagis["Tarih"].dt.year 
yagis["ay"] = yagis["Tarih"].dt.month 
 
yagis_dict = yagis.groupby(["yil","ay","LOKASYON"])["YAGIS"].mean().to_dict() 
 
finalDF["YAGIS"]  = finalDF.apply(lambda x : yagis_dict[(x["Tarih"].year,x["Tarih"].month, x["LOKASYON"])] if (pd.isna(x["YAGIS"])) else x["YAGIS"] ,axis=1)

In [65]:
finalDF

,Tarih,trafo_id,LOKASYON,ŞEBEKE_UNSURU,BAĞIL_NEM,BULUTSUZLUK_ORANI,RADYASYON,RUZGAR_HIZI,RUZGAR_YONU,SICAKLIK,YAGIS
0,2021-05-01,0,Urla,OG Fideri,0.009292,0.083333,258.153708,1.412833,3.167667,21.471917,0.000000
1,2021-05-01,1,İzmir,OG Fideri,0.005583,0.086667,276.297750,1.301917,2.800917,22.673958,0.000000
2,2021-05-01,2,İzmir,TM Fideri,0.005583,0.086667,276.297750,1.301917,2.800917,22.673958,0.000000
3,2021-05-01,3,Bergama,OG Fideri,0.005917,0.075000,301.909500,1.518750,2.785000,22.711375,0.000000
4,2021-05-01,4,İzmir,OG Fideri,0.005583,0.086667,276.297750,1.301917,2.800917,22.673958,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
26007139,2022-06-12,63762,İzmir,Abone Tesisi,0.009264,0.086667,315.413639,3.126986,1.440222,26.457375,0.007833
26007140,2022-06-12,63763,İzmir,Abone Tesisi,0.009264,0.086667,315.413639,3.126986,1.440222,26.457375,0.007833
26007141,2022-06-12,63764,İzmir,Abone Tesisi,0.009264,0.086667,315.413639,3.126986,1.440222,26.457375,0.007833
26007142,2022-06-12,63765,Akhisar,Abone Tesisi,0.009951,0.148333,303.247347,3.368187,2.050181,24.740340,0.372035


In [15]:
finalDF = pd.merge(finalDF,trainDF, on=["Tarih",'trafo_id',"LOKASYON"],how="left")
finalDF["target"].fillna(0, inplace=True)
finalDF["target"]=finalDF["target"].astype(int)
finalDF['SÜREYE_GÖRE'].fillna(0,inplace=True)

finalDF.drop("BAŞLAMA_TARİHİ_VE_ZAMANI",axis=1,inplace=True)
finalDF.drop("KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA",axis=1,inplace=True)

In [16]:
finalDF


,Tarih,trafo_id,LOKASYON,ŞEBEKE_UNSURU,BAĞIL_NEM,BULUTSUZLUK_ORANI,RADYASYON,RUZGAR_HIZI,RUZGAR_YONU,SICAKLIK,...,KESİNTİ_NO,ŞEBEKE_UNSURU_KODU,KAYNAĞA_GÖRE,SÜREYE_GÖRE,KESİNTİ_SÜRESİ,KENTSEL_AG,KENTALTI_OG,KENTALTI_AG,TOPLAM_KENTSEL_AG,target
0,2021-05-01,0,Urla,OG Fideri,0.009292,0.083333,258.153708,1.412833,3.167667,21.471917,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0
1,2021-05-01,1,İzmir,OG Fideri,0.005583,0.086667,276.297750,1.301917,2.800917,22.673958,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0
2,2021-05-01,2,İzmir,TM Fideri,0.005583,0.086667,276.297750,1.301917,2.800917,22.673958,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0
3,2021-05-01,3,Bergama,OG Fideri,0.005917,0.075000,301.909500,1.518750,2.785000,22.711375,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0
4,2021-05-01,4,İzmir,OG Fideri,0.005583,0.086667,276.297750,1.301917,2.800917,22.673958,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26010901,2022-06-12,63762,İzmir,Abone Tesisi,0.009264,0.086667,315.413639,3.126986,1.440222,26.457375,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0
26010902,2022-06-12,63763,İzmir,Abone Tesisi,0.009264,0.086667,315.413639,3.126986,1.440222,26.457375,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0
26010903,2022-06-12,63764,İzmir,Abone Tesisi,0.009264,0.086667,315.413639,3.126986,1.440222,26.457375,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0
26010904,2022-06-12,63765,Akhisar,Abone Tesisi,0.009951,0.148333,303.247347,3.368187,2.050181,24.740340,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0


In [33]:

#? trainDF["KENTSEL_AG"].value_counts()/satirSayisi # +++
#? trainDF["TOPLAM_KENTSEL_AG"].value_counts()/satirSayisi # +++
# quantillara bak estrem degerleri at modunu nanlara at

#****************

# len(finalDF[finalDF["BAĞIL_NEM"].isna()])/len(finalDF)*100 # 1.4706792604609773
# len(finalDF[finalDF["RADYASYON"].isna()])/len(finalDF)*100 # 1.4706792604609773
# len(finalDF[finalDF["RUZGAR_HIZI"].isna()])/len(finalDF)*100 # 1.4706792604609773
# len(finalDF[finalDF["RUZGAR_YONU"].isna()])/len(finalDF)*100 # 1.4706792604609773
# len(finalDF[finalDF["SICAKLIK"].isna()])/len(finalDF)*100 # 1.5054300684489808
# len(finalDF[finalDF["YAGIS"].isna()])/len(finalDF)*100 # 1.715753384368849


finalDF[(finalDF["SICAKLIK"].isna()) & (finalDF["target"]==1)].sort_values(by="Tarih") # 06.06.2022 sonrası hava durumu verisi olmayanlar için napacagız??

# finalDF.groupby(["LOKASYON","Tarih"]).mean()






,Tarih,trafo_id,LOKASYON,ŞEBEKE_UNSURU,BAĞIL_NEM,BULUTSUZLUK_ORANI,RADYASYON,RUZGAR_HIZI,RUZGAR_YONU,SICAKLIK,...,KESİNTİ_NO,ŞEBEKE_UNSURU_KODU,KAYNAĞA_GÖRE,SÜREYE_GÖRE,KESİNTİ_SÜRESİ,KENTSEL_AG,KENTALTI_OG,KENTALTI_AG,TOPLAM_KENTSEL_AG,target
9818902,2021-10-02,927,Alaşehir,KÖK,0.006167,0.0,231.593295,0.954871,5.425594,NaN,...,2055621.0,DELEMENLER KÖK 45-73-M00490_HACIALİLER M03_208...,Dağıtım-OG,1.0,0.012500,357.0,0.0,0.0,4.462500,1
9880382,2021-10-02,62427,Urla,Abone Tesisi,0.009917,0.0,225.657903,2.280875,2.667578,NaN,...,2055598.0,GÜLBAHÇE TR-4 35-19-L00144_956690301_956690301,Dağıtım-AG,2.0,1.080833,1.0,0.0,0.0,1.080833,1
9880325,2021-10-02,62370,Urla,Abone Tesisi,0.009917,0.0,225.657903,2.280875,2.667578,NaN,...,2055664.0,TR-80 35-19-L00080_956684473_956684473,Dağıtım-AG,2.0,1.435000,2.0,0.0,0.0,2.870000,1
9870471,2021-10-02,52516,Ödemiş,Abone Tesisi,0.005958,0.0,235.581027,1.187760,2.898237,NaN,...,2055763.0,GÖLCÜK TR-22 35-15-L00319_950394772_950394772,Dağıtım-AG,2.0,1.127778,1.0,0.0,0.0,1.127778,1
9870295,2021-10-02,52340,Ödemiş,Abone Tesisi,0.005958,0.0,235.581027,1.187760,2.898237,NaN,...,2055707.0,KURUCUOVA TR-3 35-15-L00254_950375925_950375925,Dağıtım-AG,2.0,2.973889,1.0,0.0,0.0,2.973889,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25952278,2022-06-12,5135,Çiğli,OG Fideri,NaN,NaN,NaN,NaN,NaN,NaN,...,2315587.0,M-1149 35-09-M01149_M-344 M03_47615662,Dağıtım-OG,1.0,0.008333,5.0,0.0,0.0,0.041667,1
25952263,2022-06-12,5120,Çiğli,OG Fideri,NaN,NaN,NaN,NaN,NaN,NaN,...,2315582.0,M-1147 GEÇİT 35-09-M01147_M-1096 M04_47553536,Dağıtım-OG,1.0,0.021667,0.0,0.0,0.0,0.000000,1
25952262,2022-06-12,5119,Çiğli,OG Fideri,NaN,NaN,NaN,NaN,NaN,NaN,...,2315102.0,M-1147 GEÇİT 35-09-M01147_M-910 M03_47553535,Dağıtım-OG,2.0,0.243333,121.0,0.0,0.0,29.443333,1
25952709,2022-06-12,5565,Manisa,OG Fideri,NaN,NaN,NaN,NaN,NaN,NaN,...,2315131.0,DUMANLAR KÖK 45-81-M00044_HatBaşıAyırıcı_53134...,Dağıtım-AG,2.0,2.528889,0.0,2.0,63.0,0.000000,1


In [51]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=1)
# y=trainDF[["TOPLAM_KENTALTI_OG","KENTSEL_OG","KIRSAL_OG","KIRSAL_AG","TOPLAM_KENTSEL_OG","TOPLAM_KENTALTI_AG","TOPLAM_KIRSAL_OG","TOPLAM_KIRSAL_AG"]] 
# y.fillna(0,inplace=True)
# x = pca.fit_transform( y)

# trainDF.drop("TOPLAM_KENTALTI_OG",axis=1,inplace=True)
# trainDF.drop("KENTSEL_OG",axis=1,inplace=True)
# trainDF.drop("KIRSAL_OG",axis=1,inplace=True)
# trainDF.drop("KIRSAL_AG",axis=1,inplace=True)
# trainDF.drop("TOPLAM_KENTSEL_OG",axis=1,inplace=True)
# trainDF.drop("TOPLAM_KENTALTI_AG",axis=1,inplace=True)
# trainDF.drop("TOPLAM_KIRSAL_OG",axis=1,inplace=True)
# trainDF.drop("TOPLAM_KIRSAL_AG",axis=1,inplace=True)



C:\Users\eminkbc\AppData\Local\Temp\ipykernel_21800\1226755286.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0,inplace=True)


In [17]:
finalDF["KENTSEL_AG"]

q1_KentselAg=finalDF["KENTSEL_AG"].quantile(q=0.25)
q3_KentselAg=finalDF["KENTSEL_AG"].quantile(q=0.75)

iqr_KentselAg = q3_KentselAg - q1_KentselAg
# KENTSEL_AG için upper ve lower bound dışında kalan degerleri buluyoruz.
ub_KentselAg = q3_KentselAg + 1.5*iqr_KentselAg
lb_KentselAg = q1_KentselAg - 1.5*iqr_KentselAg

#NAN degerleri upper ve lower bound dışında kalmayan satırların modunu alarak dolduruyoruz
modeDegeri=finalDF[finalDF["KENTSEL_AG"].apply(lambda x: 1 if x>ub_KentselAg else (1 if x<lb_KentselAg else 0))==0]["KENTSEL_AG"].mode()[0]
finalDF["KENTSEL_AG"].fillna(modeDegeri,inplace=True)


q1_KentselAg=finalDF["TOPLAM_KENTSEL_AG"].quantile(q=0.25)
q3_KentselAg=finalDF["TOPLAM_KENTSEL_AG"].quantile(q=0.75)

iqr_KentselAg = q3_KentselAg - q1_KentselAg
# TOPLAM_KENTSEL_AG için upper ve lower bound dışında kalan degerleri buluyoruz.
ub_KentselAg = q3_KentselAg + 1.5*iqr_KentselAg
lb_KentselAg = q1_KentselAg - 1.5*iqr_KentselAg

#NAN degerleri upper ve lower bound dışında kalmayan satırların modunu alarak dolduruyoruz
modeDegeri=finalDF[finalDF["TOPLAM_KENTSEL_AG"].apply(lambda x: 1 if x>ub_KentselAg else (1 if x<lb_KentselAg else 0))==0]["KENTSEL_AG"].mode()[0]
finalDF["TOPLAM_KENTSEL_AG"].fillna(modeDegeri,inplace=True)

In [41]:
finalDF[finalDF["SICAKLIK"].isnull()]


,trafo_id,LOKASYON,ŞEBEKE_UNSURU,BAĞIL_NEM,BULUTSUZLUK_ORANI,RADYASYON,RUZGAR_HIZI,RUZGAR_YONU,SICAKLIK,YAGIS,...,SÜREYE_GÖRE,KESİNTİ_SÜRESİ,KENTSEL_AG,KENTALTI_OG,KENTALTI_AG,TOPLAM_KENTSEL_AG,target,KAYNAĞA_GÖRE_Dağıtım-AG,KAYNAĞA_GÖRE_Dağıtım-OG,KAYNAĞA_GÖRE_İletim
480597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13466836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23036636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4419156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2176084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21466665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18109010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23828986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15296806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# finalDF.info()

"""           
 14  KAYNAĞA_GÖRE                       object        
   
"""
# şebeke unsur kod kolonunu gruplayıp targeta göre ortalamasını alıp kolonu sayısal bir değere dönüştürüyoruz 
sebekeUnsuruDict=finalDF.groupby("ŞEBEKE_UNSURU")["target"].mean().to_dict()
finalDF["ŞEBEKE_UNSURU"]=finalDF["ŞEBEKE_UNSURU"].apply(lambda x: sebekeUnsuruDict[x])

# lokasyon kolonunu gruplayıp targeta göre ortalamasını alıp kolonu sayısal bir değere dönüştürüyoruz 
lokasyonDict=finalDF.groupby("LOKASYON")["target"].mean().to_dict()
finalDF["LOKASYON"]=finalDF["LOKASYON"].apply(lambda x: lokasyonDict[x])

finalDF = pd.get_dummies(finalDF,columns=["KAYNAĞA_GÖRE"])

In [37]:
# del finalDF["Tarih"]
del finalDF["ŞEBEKE_UNSURU_KODU"]


In [50]:
finalDF

,Tarih,trafo_id,LOKASYON,ŞEBEKE_UNSURU
0,2021-05-01,0,Urla,OG Fideri
1,2021-05-01,1,İzmir,OG Fideri
2,2021-05-01,2,İzmir,TM Fideri
3,2021-05-01,3,Bergama,OG Fideri
4,2021-05-01,4,İzmir,OG Fideri
...,...,...,...,...
26007139,2022-06-12,63762,İzmir,Abone Tesisi
26007140,2022-06-12,63763,İzmir,Abone Tesisi
26007141,2022-06-12,63764,İzmir,Abone Tesisi
26007142,2022-06-12,63765,Akhisar,Abone Tesisi


In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
finalDF = finalDF.sample(frac = 0.5)
limit = int(len(finalDF) * 0.70)
train = finalDF[:limit]
test = finalDF[limit:]

train_y = train["target"]
del train["target"]

test_y = test["target"]
del test["target"]

clf = RandomForestClassifier()
clf.fit( train, train_y )
print( clf.score( test, test_y ) )
print( "f1_score", f1_score(test_y, clf.predict(test)) )

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
# trafo_date=submissionDF["trafo_id_date"].str.split("__",expand=True)
# submissionDF["trafo_id"]  = trafo_date[0]
# submissionDF["Tarih"]  = trafo_date[1]
# submissionDF.drop("trafo_id_date",axis=1,inplace=True)